# 영어 Word2Vec 만들기

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
import re, zipfile
from urllib.request import urlretrieve
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize

### 훈련 데이터 이해하기

In [3]:
urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/09.%20Word%20Embedding/dataset/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7f04d671acd0>)

In [4]:
!ls -l ted*

-rw-r--r-- 1 root root 74533638 Jan 25 01:05 ted_en-20160408.xml


In [5]:
!more ted*

<?xml version="1.0" encoding="UTF-8"?>
<xml language="en"><file id="1">
  <head>
    <url>http://www.ted.com/talks/knut_haanaes_two_reasons_companies_fail_and_ho
w_to_avoid_them</url>
    <pagesize>72832</pagesize>
    <dtime>Fri Apr 01 00:57:03 CEST 2016</dtime>
    <encoding>UTF-8</encoding>
    <content-type>text/html; charset=utf-8</content-type>
    <keywords>talks, business, creativity, curiosity, goal-setting, innovation, 
motivation, potential, success, work</keywords>
    <speaker>Knut Haanaes</speaker>
    <talkid>2470</talkid>
    <videourl>http://download.ted.com/talks/KnutHaanaes_2015S.mp4</videourl>
    <videopath>talks/KnutHaanaes_2015S.mp4</videopath>
    <date>2015/06/30</date>
    <title>Knut Haanaes: Two reasons companies fail -- and how to avoid them</ti
tle>
    <description>TED Talk Subtitles and Transcript: Is it possible to run a comp
any and reinvent it at the same time? For business strategist Knut Haanaes, the 
ability to innovate after becoming successful is

### 훈련 데이터 전처리하기

In [6]:
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

In [7]:
# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

In [8]:
# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

In [9]:
# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

In [10]:
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

In [11]:
# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

In [13]:
print(f'총 샘플의 개수 : {len(result):,d}')

총 샘플의 개수 : 273,424


In [14]:
# 샘플 3개만 출력
for line in result[:3]:
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


### Word2Vec 훈련시키기
    size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.
    window = 컨텍스트 윈도우 크기
    min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
    workers = 학습을 위한 프로세스 수
    sg = 0은 CBOW, 1은 Skip-gram.

In [15]:
from gensim.models import Word2Vec

In [17]:
%time model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)

CPU times: user 1min 5s, sys: 445 ms, total: 1min 6s
Wall time: 38.6 s


In [18]:
# man과 자주 나오는(similar) 단어 10개
model.wv.most_similar("man")

[('woman', 0.8553561568260193),
 ('guy', 0.8172657489776611),
 ('boy', 0.7913240790367126),
 ('lady', 0.783963680267334),
 ('girl', 0.7454286813735962),
 ('gentleman', 0.7235683798789978),
 ('soldier', 0.7101308107376099),
 ('poet', 0.6955063343048096),
 ('kid', 0.6814377307891846),
 ('friend', 0.668156623840332)]

In [19]:
# man - boy + girl --> woman
model.wv.most_similar(positive=['man','girl'], negative=['boy'], topn=3)

[('woman', 0.8573539853096008),
 ('lady', 0.7575744390487671),
 ('guy', 0.7471429705619812)]

### 모델 저장하고 로드하기

In [20]:
model.wv.save_word2vec_format('eng_w2v') # 모델 저장

In [21]:
!ls -l

total 97864
-rw-r--r-- 1 root root 25672113 Jan 25 01:30 eng_w2v
drwxr-xr-x 1 root root     4096 Jan  7 14:33 sample_data
-rw-r--r-- 1 root root 74533638 Jan 25 01:05 ted_en-20160408.xml


In [22]:
from gensim.models import KeyedVectors

loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드
loaded_model.most_similar("man")

[('woman', 0.8553561568260193),
 ('guy', 0.8172657489776611),
 ('boy', 0.7913240790367126),
 ('lady', 0.783963680267334),
 ('girl', 0.7454286813735962),
 ('gentleman', 0.7235683798789978),
 ('soldier', 0.7101308107376099),
 ('poet', 0.6955063343048096),
 ('kid', 0.6814377307891846),
 ('friend', 0.668156623840332)]